# Loading C++ Models: JAX-Compatible vs FFI Approaches

This notebook demonstrates two approaches for loading user-defined C++ phase-type models:
1. **JAX-Compatible** (default): Full JAX support with automatic differentiation
2. **FFI Approach**: Build once, use many times for maximum efficiency

We'll use the **Rabbit Flooding Model** as our example - a fun simulation where rabbits hop between islands that can flood.

In [ ]:
# Import required libraries
import numpy as np
import jax
import jax.numpy as jnp
from ptdalgorithms import Graph
import matplotlib.pyplot as plt
import time
from IPython.display import display, Markdown

# Set up JAX
import os
os.environ['JAX_ENABLE_X64'] = 'True'

print(f"JAX version: {jax.__version__}")
print(f"NumPy version: {np.__version__}")

## The Rabbit Flooding Model

This model simulates:
- Rabbits starting on the left island
- Rabbits hop between islands at rate 1.0
- Islands can flood, eliminating all rabbits on that island
- The process ends when all rabbits are eliminated (absorbed)

**Parameters:**
1. Number of initial rabbits
2. Flood rate for left island
3. Flood rate for right island

## Approach 1: JAX-Compatible (Default)

This approach provides full JAX support including JIT compilation, automatic differentiation, and vectorization.

In [ ]:
# Load the model with JAX support (default)
rabbit_model_jax = Graph.load_cpp_model("user_models/rabbit_flooding.cpp")

print("✅ JAX-compatible model loaded")
print("Type:", type(rabbit_model_jax))
print("\nThis is a callable function that:")
print("- Takes parameters and times as JAX arrays")
print("- Returns PDF values as JAX arrays")
print("- Rebuilds the graph on each call (for JAX purity)")

In [ ]:
# Basic usage with JAX
params_jax = jnp.array([3.0, 0.5, 0.5])  # 3 rabbits, symmetric flooding
times_jax = jnp.linspace(0.1, 10.0, 50)

# Compute PDF
pdf_jax = rabbit_model_jax(params_jax, times_jax)

print(f"Parameters: {int(params_jax[0])} rabbits, flood rates = {params_jax[1]:.1f}, {params_jax[2]:.1f}")
print(f"PDF shape: {pdf_jax.shape}")
print(f"Total absorption probability: {jnp.sum(pdf_jax):.4f}")
print(f"Mean time to absorption: {jnp.sum(times_jax * pdf_jax) / jnp.sum(pdf_jax):.4f}")

In [ ]:
# JIT Compilation for Speed
jit_model = jax.jit(rabbit_model_jax)

# First call compiles
%time pdf_compiled = jit_model(params_jax, times_jax)

# Subsequent calls are fast
%time pdf_compiled = jit_model(params_jax, times_jax)

print(f"\n✅ JIT compilation successful")
print(f"Results match: {jnp.allclose(pdf_jax, pdf_compiled)}")

In [ ]:
# Automatic Differentiation
def survival_probability(params):
    """Compute total probability of absorption by t=10"""
    pdf = rabbit_model_jax(params, times_jax)
    return jnp.sum(pdf)

# Compute gradient with respect to parameters
grad_fn = jax.grad(survival_probability)
gradients = grad_fn(params_jax)

print("Sensitivity Analysis (gradients):")
print(f"  ∂P/∂(num_rabbits):    {gradients[0]:.6f}")
print(f"  ∂P/∂(flood_left):     {gradients[1]:.6f}")
print(f"  ∂P/∂(flood_right):    {gradients[2]:.6f}")
print("\nInterpretation:")
print("- Positive gradient for flood rates means higher flood rate increases absorption")
print("- This makes sense: more flooding = rabbits eliminated faster")

In [ ]:
# Vectorization with vmap
# Test different numbers of rabbits
rabbit_counts = jnp.array([
    [2.0, 0.5, 0.5],
    [3.0, 0.5, 0.5],
    [4.0, 0.5, 0.5],
    [5.0, 0.5, 0.5],
])

# Vectorize over parameter sets
vmap_model = jax.vmap(lambda p: rabbit_model_jax(p, times_jax))
pdf_batch = vmap_model(rabbit_counts)

print("Batch processing with vmap:")
print(f"Input shape: {rabbit_counts.shape} (4 parameter sets)")
print(f"Output shape: {pdf_batch.shape} (4 × 50 PDFs)")
print("\nAbsorption probabilities:")
for i, params in enumerate(rabbit_counts):
    prob = jnp.sum(pdf_batch[i])
    print(f"  {int(params[0])} rabbits: {prob:.4f}")

In [ ]:
# Parallelization with pmap (for multi-device systems)
devices = jax.devices()
n_devices = len(devices)
print(f"Available devices: {devices}")
print(f"Number of devices: {n_devices}")

if n_devices > 1:
    # Multiple GPUs/TPUs available - distribute across devices
    print("\nUsing pmap for multi-device parallelization:")
    
    # Parameters for each device
    params_per_device = jnp.array([
        [2.0 + i, 0.5, 0.5] for i in range(n_devices)
    ])
    
    # Parallel map across devices
    pmap_model = jax.pmap(lambda p: rabbit_model_jax(p, times_jax))
    pdf_parallel = pmap_model(params_per_device)
    
    print(f"✅ pmap successful across {n_devices} devices")
    print(f"   Output shape: {pdf_parallel.shape}")
    
    # Parallel gradients
    pmap_grad = jax.pmap(jax.grad(lambda p: jnp.sum(rabbit_model_jax(p, times_jax))))
    parallel_grads = pmap_grad(params_per_device)
    print(f"✅ Parallel gradients: shape={parallel_grads.shape}")
else:
    # Single device (CPU or single GPU)
    print("\nSingle device system - pmap will use thread-level parallelism")
    print("Note: On multi-GPU/TPU systems, pmap distributes across devices")
    
    # Even with 1 device, pmap works
    params_single = jnp.array([[3.0, 0.5, 0.5]])
    pmap_model = jax.pmap(lambda p: rabbit_model_jax(p, times_jax))
    pdf_pmap = pmap_model(params_single)
    print(f"✅ pmap works: shape={pdf_pmap.shape}")

print("\nSummary of JAX transformations:")
print("✅ jit  - JIT compilation for speed")
print("✅ grad - Automatic differentiation")
print("✅ vmap - Vectorization within device")
print("✅ pmap - Parallelization across devices")

## Approach 2: FFI (Foreign Function Interface)

This approach separates graph construction from computation, allowing you to build once and use many times.

In [ ]:
# Load the model with FFI approach
rabbit_builder = Graph.load_cpp_model("user_models/rabbit_flooding.cpp", use_ffi=True)

print("✅ FFI builder loaded")
print("Type:", type(rabbit_builder))
print("\nThis is a builder function that:")
print("- Takes parameters and returns a Graph object")
print("- Graph can be reused many times without rebuilding")
print("- Much faster for repeated evaluations")

In [ ]:
# Build a graph with specific parameters
params_ffi = np.array([3.0, 0.5, 0.5])  # Same as JAX example
rabbit_graph = rabbit_builder(params_ffi)

print(f"✅ Graph built with {int(params_ffi[0])} rabbits")
print(f"Graph type: {type(rabbit_graph)}")
print("\nNow we can use this graph many times without rebuilding:")

In [ ]:
# Use the same graph for multiple evaluations
times_ffi = np.linspace(0.1, 10.0, 50)

# Compute PDF at all times (no rebuilding!)
pdf_ffi = np.array([rabbit_graph.pdf(t, 100) for t in times_ffi])

print(f"PDF computed at {len(times_ffi)} time points")
print(f"Total absorption probability: {np.sum(pdf_ffi):.4f}")
print(f"Mean time to absorption: {np.sum(times_ffi * pdf_ffi) / np.sum(pdf_ffi):.4f}")
print("\nNote: Same results as JAX approach!")

In [ ]:
# Performance comparison
n_evaluations = 1000
test_times = np.random.uniform(0.1, 10.0, n_evaluations)

# FFI approach: Use pre-built graph
start = time.time()
for t in test_times:
    pdf = rabbit_graph.pdf(t, 100)
ffi_time = time.time() - start

# JAX approach: Rebuilds graph each time
start = time.time()
for i in range(min(100, n_evaluations)):  # Only 100 to avoid long wait
    pdf = rabbit_model_jax(params_jax, jnp.array([test_times[i]]))
jax_time_100 = time.time() - start
jax_time_estimated = jax_time_100 * (n_evaluations / 100)

print(f"Performance for {n_evaluations} evaluations:")
print(f"  FFI approach: {ffi_time:.4f} seconds")
print(f"  JAX approach (estimated): {jax_time_estimated:.4f} seconds")
print(f"  Speedup: {jax_time_estimated/ffi_time:.1f}x")

In [ ]:
# Parameter sweep with FFI
# Test asymmetric flooding
flood_rates_left = np.linspace(0.1, 2.0, 10)
results = []

for flood_left in flood_rates_left:
    # Build new graph for these parameters
    g = rabbit_builder(np.array([3.0, flood_left, 0.5]))
    
    # Compute mean absorption time
    pdf_vals = np.array([g.pdf(t, 100) for t in times_ffi])
    mean_time = np.sum(times_ffi * pdf_vals) / np.sum(pdf_vals)
    results.append(mean_time)

print(f"Parameter sweep completed: {len(flood_rates_left)} different flood rates")
print(f"Mean absorption times range: [{min(results):.2f}, {max(results):.2f}]")

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Plot 1: PDF comparison
ax1 = axes[0, 0]
ax1.plot(times_ffi, pdf_ffi, 'b-', label='FFI', linewidth=2)
ax1.plot(times_jax, pdf_jax, 'r--', label='JAX', linewidth=2, alpha=0.7)
ax1.set_xlabel('Time')
ax1.set_ylabel('PDF')
ax1.set_title('PDF Comparison (Both Approaches)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Survival curve
ax2 = axes[0, 1]
cdf = np.cumsum(pdf_ffi) * (times_ffi[1] - times_ffi[0])
survival = 1 - cdf
ax2.plot(times_ffi, survival, 'g-', linewidth=2)
ax2.fill_between(times_ffi, 0, survival, alpha=0.3, color='green')
ax2.set_xlabel('Time')
ax2.set_ylabel('Survival Probability')
ax2.set_title('Rabbit Survival Over Time')
ax2.grid(True, alpha=0.3)

# Plot 3: Performance comparison
ax3 = axes[0, 2]
methods = ['JAX\n(rebuilds)', 'FFI\n(reuses)']
times_comp = [jax_time_estimated, ffi_time]
colors = ['red', 'blue']
bars = ax3.bar(methods, times_comp, color=colors, alpha=0.7)
ax3.set_ylabel('Time (seconds)')
ax3.set_title(f'Performance: {n_evaluations} Evaluations')
for bar, t in zip(bars, times_comp):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{t:.3f}s', ha='center')

# Plot 4: Effect of rabbit count (using vmap results)
ax4 = axes[1, 0]
rabbit_nums = [2, 3, 4, 5]
for i, n in enumerate(rabbit_nums):
    ax4.plot(times_jax, pdf_batch[i], label=f'{n} rabbits', alpha=0.7)
ax4.set_xlabel('Time')
ax4.set_ylabel('PDF')
ax4.set_title('Effect of Initial Rabbit Count (vmap)')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: Flood rate sensitivity
ax5 = axes[1, 1]
ax5.plot(flood_rates_left, results, 'purple', linewidth=2, marker='o')
ax5.set_xlabel('Left Island Flood Rate')
ax5.set_ylabel('Mean Time to Absorption')
ax5.set_title('Sensitivity to Flood Rate (FFI sweep)')
ax5.grid(True, alpha=0.3)

# Plot 6: Feature comparison table
ax6 = axes[1, 2]
ax6.axis('off')
table_data = [
    ['Feature', 'JAX', 'FFI'],
    ['JIT', '✅', '❌'],
    ['Gradients', '✅', '❌'],
    ['vmap', '✅', '❌'],
    ['pmap', '✅', '❌'],
    ['Graph reuse', '❌', '✅'],
    ['Speed (fixed θ)', '🐢', '🚀'],
]
table = ax6.table(cellText=table_data, cellLoc='center', loc='center',
                  colWidths=[0.3, 0.2, 0.2])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
ax6.set_title('Feature Comparison', pad=20)

plt.suptitle('Rabbit Flooding Model: JAX vs FFI Approaches', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Plot 1: PDF comparison
ax1 = axes[0, 0]
ax1.plot(times_ffi, pdf_ffi, 'b-', label='FFI', linewidth=2)
ax1.plot(times_jax, pdf_jax, 'r--', label='JAX', linewidth=2, alpha=0.7)
ax1.set_xlabel('Time')
ax1.set_ylabel('PDF')
ax1.set_title('PDF Comparison (Both Approaches)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Survival curve
ax2 = axes[0, 1]
cdf = np.cumsum(pdf_ffi) * (times_ffi[1] - times_ffi[0])
survival = 1 - cdf
ax2.plot(times_ffi, survival, 'g-', linewidth=2)
ax2.fill_between(times_ffi, 0, survival, alpha=0.3, color='green')
ax2.set_xlabel('Time')
ax2.set_ylabel('Survival Probability')
ax2.set_title('Rabbit Survival Over Time')
ax2.grid(True, alpha=0.3)

# Plot 3: Performance comparison
ax3 = axes[0, 2]
methods = ['JAX\n(rebuilds)', 'FFI\n(reuses)']
times_comp = [jax_time_estimated, ffi_time]
colors = ['red', 'blue']
bars = ax3.bar(methods, times_comp, color=colors, alpha=0.7)
ax3.set_ylabel('Time (seconds)')
ax3.set_title(f'Performance: {n_evaluations} Evaluations')
for bar, t in zip(bars, times_comp):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{t:.3f}s', ha='center')

# Plot 4: Effect of rabbit count (using vmap results)
ax4 = axes[1, 0]
rabbit_nums = [2, 3, 4, 5]
for i, n in enumerate(rabbit_nums):
    ax4.plot(times_jax, pdf_batch[i], label=f'{n} rabbits', alpha=0.7)
ax4.set_xlabel('Time')
ax4.set_ylabel('PDF')
ax4.set_title('Effect of Initial Rabbit Count')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: Flood rate sensitivity
ax5 = axes[1, 1]
ax5.plot(flood_rates_left, results, 'purple', linewidth=2, marker='o')
ax5.set_xlabel('Left Island Flood Rate')
ax5.set_ylabel('Mean Time to Absorption')
ax5.set_title('Sensitivity to Flood Rate (FFI sweep)')
ax5.grid(True, alpha=0.3)

# Plot 6: Feature comparison table
ax6 = axes[1, 2]
ax6.axis('off')
table_data = [
    ['Feature', 'JAX', 'FFI'],
    ['JIT', '✅', '❌'],
    ['Gradients', '✅', '❌'],
    ['vmap', '✅', '❌'],
    ['Graph reuse', '❌', '✅'],
    ['Speed (fixed θ)', '🐢', '🚀'],
]
table = ax6.table(cellText=table_data, cellLoc='center', loc='center',
                  colWidths=[0.3, 0.2, 0.2])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)
ax6.set_title('Feature Comparison', pad=20)

plt.suptitle('Rabbit Flooding Model: JAX vs FFI Approaches', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## Practical Example: Parameter Optimization

Let's find flood rates that achieve a target mean absorption time.

In [ ]:
# Using JAX for gradient-based optimization
target_time = 3.0  # Target mean absorption time

def loss_function(flood_rates):
    """Loss: squared difference from target mean time"""
    params = jnp.array([3.0, flood_rates[0], flood_rates[1]])
    pdf = rabbit_model_jax(params, times_jax)
    mean_time = jnp.sum(times_jax * pdf) / jnp.sum(pdf)
    return (mean_time - target_time) ** 2

# Optimize using gradient descent
flood_rates = jnp.array([1.0, 1.0])  # Initial guess
learning_rate = 0.1

print("Optimizing flood rates to achieve target mean time...")
for i in range(20):
    loss, grad = jax.value_and_grad(loss_function)(flood_rates)
    flood_rates = flood_rates - learning_rate * grad
    
    if i % 5 == 0:
        print(f"Iteration {i:2d}: Loss = {loss:.6f}, Rates = [{flood_rates[0]:.3f}, {flood_rates[1]:.3f}]")

# Verify result
final_params = jnp.array([3.0, flood_rates[0], flood_rates[1]])
final_pdf = rabbit_model_jax(final_params, times_jax)
final_mean = jnp.sum(times_jax * final_pdf) / jnp.sum(final_pdf)

print(f"\n✅ Optimization complete!")
print(f"Target mean time: {target_time:.2f}")
print(f"Achieved mean time: {final_mean:.2f}")
print(f"Optimal flood rates: [{flood_rates[0]:.3f}, {flood_rates[1]:.3f}]")

## Summary and Recommendations

### When to use each approach:

In [ ]:
summary = """
## 🎯 Quick Decision Guide

### Use JAX-Compatible (default) when:
- ✅ You need automatic differentiation (gradients)
- ✅ Doing parameter optimization
- ✅ Integrating with JAX/ML libraries
- ✅ Research and experimentation
- ✅ Parameters change frequently

```python
model = Graph.load_cpp_model("model.cpp")
pdf = model(params, times)  # Full JAX support
```

### Use FFI Approach when:
- ✅ Parameters are fixed or change rarely
- ✅ Need maximum performance
- ✅ Monte Carlo simulations
- ✅ Real-time/production systems
- ✅ Evaluating same model thousands of times

```python
builder = Graph.load_cpp_model("model.cpp", use_ffi=True)
graph = builder(params)  # Build once
pdf = graph.pdf(t, 100)  # Use many times
```

### Performance Summary:
- JAX approach: ~{:.2f} seconds for {} evaluations
- FFI approach: ~{:.2f} seconds for {} evaluations
- **Speedup: {:.0f}x faster with FFI for repeated evaluations**

### Key Insight:
Both approaches use the **same C++ model files**! You can switch between them by
changing a single parameter: `use_ffi=True`
""".format(jax_time_estimated, n_evaluations, ffi_time, n_evaluations, 
           jax_time_estimated/ffi_time)

display(Markdown(summary))

## Try It Yourself!

Experiment with different parameters and see how the rabbits' fate changes:

In [ ]:
# Interactive example - modify these parameters!
your_params = np.array([
    5.0,   # Number of rabbits (try 1-10)
    0.3,   # Left island flood rate (try 0.1-2.0)
    1.5,   # Right island flood rate (try 0.1-2.0)
])

# Build your graph
your_graph = rabbit_builder(your_params)

# Compute survival curve
your_times = np.linspace(0.1, 15.0, 150)
your_pdf = np.array([your_graph.pdf(t, 100) for t in your_times])
your_cdf = np.cumsum(your_pdf) * (your_times[1] - your_times[0])
your_survival = 1 - your_cdf

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(your_times, your_pdf, 'b-', linewidth=2)
ax1.fill_between(your_times, 0, your_pdf, alpha=0.3)
ax1.set_xlabel('Time')
ax1.set_ylabel('PDF')
ax1.set_title(f'Absorption Time Distribution ({int(your_params[0])} rabbits)')
ax1.grid(True, alpha=0.3)

ax2.plot(your_times, your_survival, 'g-', linewidth=2)
ax2.fill_between(your_times, 0, your_survival, alpha=0.3, color='green')
ax2.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='50% survival')
ax2.set_xlabel('Time')
ax2.set_ylabel('Survival Probability')
ax2.set_title(f'Flood rates: Left={your_params[1]:.1f}, Right={your_params[2]:.1f}')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
mean_time = np.sum(your_times * your_pdf) / np.sum(your_pdf)
median_idx = np.argmax(your_cdf >= 0.5)
median_time = your_times[median_idx] if median_idx > 0 else np.inf

print(f"\n📊 Statistics for your configuration:")
print(f"  Rabbits: {int(your_params[0])}")
print(f"  Flood rates: Left={your_params[1]:.2f}, Right={your_params[2]:.2f}")
print(f"  Mean time to absorption: {mean_time:.2f}")
print(f"  Median time to absorption: {median_time:.2f}")
print(f"  Probability of absorption by t=10: {your_cdf[np.argmin(np.abs(your_times - 10))]:.3f}")

if your_params[1] < your_params[2]:
    print("  💡 Tip: Left island is safer (lower flood rate)")
elif your_params[1] > your_params[2]:
    print("  💡 Tip: Right island is safer (lower flood rate)")
else:
    print("  💡 Tip: Both islands have equal flood risk")